# Machine Learning Project 1  ( Part 1 )
## UNICAMP

## Dataset 1

Load Datasets


In [21]:
import pandas as pd
import numpy as np

df = pd.read_table("cluster.dat",header=None, sep="\s+")
datos = pd.DataFrame(df)

datos


,0,1
0,1555.0,28.65
1,1490.0,27.55
2,1445.0,28.35
3,1415.0,28.80
4,1375.0,28.05
...,...,...
568,3420.0,27.95
569,3465.0,26.85
570,3525.0,26.00
571,3570.0,26.15


Transformamos el dataset a un array de nummpy

In [19]:
datos=datos.to_numpy()
datos

array([[1555.  ,   28.65],
       [1490.  ,   27.55],
       [1445.  ,   28.35],
       ...,
       [3525.  ,   26.  ],
       [3570.  ,   26.15],
       [3440.  ,   25.6 ]])

Shuffle data

In [20]:
np.random.shuffle(datos)
datos

array([[1135.  ,    5.55],
       [ 920.  ,   23.9 ],
       [3240.  ,   27.1 ],
       ...,
       [1190.  ,   24.5 ],
       [1300.  ,   29.15],
       [3570.  ,   26.15]])

Split data in train and test:
- 90% train 
- 10% test

In [29]:
datos_train = datos[:int(0.9*len(datos))]
datos_test = datos[int(0.9*len(datos)):]

print("Train", datos_train.shape)
print("Test", datos_test.shape)

Train (515, 2)
Test (58, 2)
